In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

c:\Users\tangu\Anaconda\envs\Deep\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Définition des transformations pour les données à partir imageNet
transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Définition des transformations pour les données
transform2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4859, 0.4996, 0.4318], std=[0.2250, 0.2203, 0.2585]),
])

In [3]:
batch_len = 32
transform = transform1

# Chargement des données
train_dataset_1 = torchvision.datasets.ImageFolder('./data/train/', transform=transform)
test_dataset_1 = torchvision.datasets.ImageFolder('./data/test/', transform=transform)

train_loader_1 = torch.utils.data.DataLoader(train_dataset_1, batch_size=batch_len, shuffle=True)
test_loader_1 = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_len, shuffle=False)

In [4]:

# Définition du modèle
model_b = models.resnet18(weights='IMAGENET1K_V1')
# Geler les paramètres de convolution
for param in model_b.parameters():
    param.requires_grad = False

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_b.fc.in_features
model_b.fc = nn.Linear(num_ftrs, 200)



# Définition du modèle
model_a = models.resnet18(weights=None)

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_a.fc.in_features
model_a.fc = nn.Linear(num_ftrs, 200)




# Définition du modèle
model_c = models.resnet18(weights='IMAGENET1K_V1')

for name, param in model_c.named_parameters():
    if 'layer1' in name:
        param.requires_grad = False
        print(name)

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_c.fc.in_features
model_c.fc = nn.Linear(num_ftrs, 200)



# Définition du modèle
model_d = models.resnet18(weights='IMAGENET1K_V1')

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_d.fc.in_features
model_d.fc = nn.Linear(num_ftrs, 200)



layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias


In [5]:
print(model_a)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=5, 
                flag1=False, save_dir='./saved_models',device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                name=''):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    model.to(device)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Sauvegarde du modèle à chaque époque
        model_name = f'model_epoch_{name}_{epoch+1}.pt'
        model_path = os.path.join(save_dir, model_name)
        torch.save(model.state_dict(), model_path)

        # if epoch == 0:
        #     if flag1:
        #         plt.imshow(inputs[0].permute(1, 2, 0))
        #         plt.show()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def test_model(model, test_loader,device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):

    model.to(device)
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")


In [7]:
def my_test (model,train_loader, test_loader, lr=0.01,num_epochs=5,name=''):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=lr)
    
    train_model(model, train_loader, criterion, optimizer,num_epochs=num_epochs,name=name)
    test_model(model, test_loader)

test pour normalisation imagenet

In [8]:
my_test(model_a,train_loader_1,test_loader_1,name="a1")

 20%|██        | 1/5 [13:11<52:46, 791.61s/it]

Epoch 1/5, Loss: 8.315707022927024


 40%|████      | 2/5 [23:19<34:09, 683.28s/it]

Epoch 2/5, Loss: 7.361970468000933


 60%|██████    | 3/5 [31:15<19:38, 589.05s/it]

...

 80%|████████  | 4/5 [39:19<09:07, 547.46s/it]

Epoch 4/5, Loss: 7.000037562630394


100%|██████████| 5/5 [54:29<00:00, 653.86s/it]

Epoch 5/5, Loss: 6.815502532612194


Accuracy: 0.008333333333333333


In [9]:
test_model(model_a, train_loader_1)

Accuracy: 0.021051433773327265


In [10]:
my_test(model_b,train_loader_1,test_loader_1,name="b1")

 20%|██        | 1/5 [07:25<29:40, 445.03s/it]

Epoch 1/5, Loss: 4.911893880150535


 40%|████      | 2/5 [13:19<19:35, 391.85s/it]

Epoch 2/5, Loss: 2.857129772793163


 60%|██████    | 3/5 [18:28<11:48, 354.05s/it]

...

 80%|████████  | 4/5 [25:41<06:25, 385.10s/it]

Epoch 4/5, Loss: 1.9711669471047142


100%|██████████| 5/5 [33:12<00:00, 398.43s/it]

Epoch 5/5, Loss: 1.9666117884896017


Accuracy: 0.44333333333333336


In [11]:
test_model(model_b, train_loader_1)

Accuracy: 0.7816340464269458


In [12]:
my_test(model_c,train_loader_1,test_loader_1,name='c1')

 20%|██        | 1/5 [12:42<50:48, 762.06s/it]

Epoch 1/5, Loss: 4.889281963868575


 40%|████      | 2/5 [24:14<36:03, 721.15s/it]

Epoch 2/5, Loss: 2.8282911729812623


 60%|██████    | 3/5 [31:18<19:30, 585.50s/it]

...

 80%|████████  | 4/5 [38:21<08:41, 521.32s/it]

Epoch 4/5, Loss: 1.9407381812008944


100%|██████████| 5/5 [45:26<00:00, 545.31s/it]

Epoch 5/5, Loss: 1.7952581756765191


Accuracy: 0.441


In [13]:
test_model(model_c, train_loader_1)

Accuracy: 0.7561447428311333


In [14]:
my_test(model_d,train_loader_1,test_loader_1,name='d1')

 20%|██        | 1/5 [11:22<45:31, 682.95s/it]

Epoch 1/5, Loss: 4.990280448740179


 40%|████      | 2/5 [18:42<26:58, 539.48s/it]

Epoch 2/5, Loss: 2.803528365655379


 60%|██████    | 3/5 [26:32<16:55, 507.76s/it]

...

 80%|████████  | 4/5 [40:03<10:27, 627.51s/it]

Epoch 4/5, Loss: 2.0774154270779004


100%|██████████| 5/5 [52:06<00:00, 625.30s/it]

Epoch 5/5, Loss: 1.7590709017623554


Accuracy: 0.44133333333333336


In [15]:
test_model(model_d, train_loader_1)

Accuracy: 0.7992717341829768


------------------------------------

In [16]:
batch_len = 32
transform = transform2

# Chargement des données
train_dataset_2 = torchvision.datasets.ImageFolder('./data/train/', transform=transform)
test_dataset_2 = torchvision.datasets.ImageFolder('./data/test/', transform=transform)

train_loader_2 = torch.utils.data.DataLoader(train_dataset_2, batch_size=batch_len, shuffle=True)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=batch_len, shuffle=False)

In [17]:

# Définition du modèle
model_b2 = models.resnet18(weights='IMAGENET1K_V1')
# Geler les paramètres de convolution
for param in model_b2.parameters():
    param.requires_grad = False

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_b2.fc.in_features
model_b2.fc = nn.Linear(num_ftrs, 200)



# Définition du modèle
model_a2 = models.resnet18(weights=None)

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_a2.fc.in_features
model_a2.fc = nn.Linear(num_ftrs, 200)




# Définition du modèle
model_c2 = models.resnet18(weights='IMAGENET1K_V1')

for name, param in model_c2.named_parameters():
    if 'layer1' in name:
        param.requires_grad = False
        print(name)

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_c2.fc.in_features
model_c2.fc = nn.Linear(num_ftrs, 200)



# Définition du modèle
model_d2 = models.resnet18(weights='IMAGENET1K_V1')

# Modifier la dernière couche pour avoir le bon nombre de classes (200 pour CUB-200)
num_ftrs = model_d2.fc.in_features
model_d2.fc = nn.Linear(num_ftrs, 200)



layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias


In [18]:
my_test(model_a2,train_loader_2,test_loader_2,name='a2')

 20%|██        | 1/5 [08:36<34:24, 516.06s/it]

Epoch 1/5, Loss: 8.129022442210804


 40%|████      | 2/5 [17:45<26:47, 535.73s/it]

Epoch 2/5, Loss: 7.395322296836159


 60%|██████    | 3/5 [26:04<17:17, 518.88s/it]

...

 80%|████████  | 4/5 [34:17<08:28, 508.52s/it]

Epoch 4/5, Loss: 7.14362457448786


100%|██████████| 5/5 [42:14<00:00, 506.93s/it]

Epoch 5/5, Loss: 6.913860100832853


Accuracy: 0.018666666666666668


In [19]:
test_model(model_a2, train_loader_2)

Accuracy: 0.05211652253072371


In [20]:
my_test(model_b2,train_loader_2,test_loader_2,name='b2')

 20%|██        | 1/5 [06:14<24:57, 374.41s/it]

Epoch 1/5, Loss: 5.010095578973943


 40%|████      | 2/5 [13:24<20:21, 407.12s/it]

Epoch 2/5, Loss: 2.9357127592780374


 60%|██████    | 3/5 [20:51<14:10, 425.46s/it]

...

 80%|████████  | 4/5 [28:17<07:13, 433.31s/it]

Epoch 4/5, Loss: 2.1385292222283105


100%|██████████| 5/5 [35:43<00:00, 428.67s/it]

Epoch 5/5, Loss: 1.7470034795457667


Accuracy: 0.44866666666666666


In [21]:
test_model(model_b2, train_loader_2)

Accuracy: 0.7938097405553027


In [22]:
my_test(model_c2,train_loader_2,test_loader_2,name='c2')

 20%|██        | 1/5 [10:30<42:00, 630.22s/it]

Epoch 1/5, Loss: 4.909759358492765


 40%|████      | 2/5 [26:36<41:23, 827.68s/it]

Epoch 2/5, Loss: 2.831424737410112


 60%|██████    | 3/5 [33:31<21:18, 639.25s/it]

...

 80%|████████  | 4/5 [44:08<10:38, 638.61s/it]

Epoch 4/5, Loss: 2.047063680562106


100%|██████████| 5/5 [1:06:32<00:00, 798.43s/it]

Epoch 5/5, Loss: 1.8416841732371938


Accuracy: 0.44366666666666665


In [23]:
test_model(model_c2, train_loader_2)

Accuracy: 0.7974510696404188


In [24]:
my_test(model_d2,train_loader_2,test_loader_2,name='d2')

 20%|██        | 1/5 [14:41<58:44, 881.20s/it]

Epoch 1/5, Loss: 5.0924355073408645


 40%|████      | 2/5 [23:54<34:24, 688.15s/it]

Epoch 2/5, Loss: 2.7908382086320356


 60%|██████    | 3/5 [41:30<28:32, 856.19s/it]

...

 80%|████████  | 4/5 [59:53<15:53, 953.56s/it]

Epoch 4/5, Loss: 2.057856057990681


100%|██████████| 5/5 [1:10:13<00:00, 842.70s/it]

Epoch 5/5, Loss: 1.8949366554346951


Accuracy: 0.43233333333333335


In [25]:
test_model(model_d2, train_loader_2)

Accuracy: 0.7543240782885753
